In [5]:
import time
import re
from bs4 import BeautifulSoup #use beautifulsoup to scrape data
from selenium import webdriver


url = "http://millercenter.org/the-presidency/presidential-speeches"

browser = webdriver.Chrome() #change this line depending on your system
browser.get(url) #go to the website 


#This is what we will change if we want the whole data. Right now it only runs for 44th president

for i in range(25,30):
    
    absolutepath_president = f'/html/body/div[2]/div/main/div[2]/div/div[2]/article/div/div[2]/div/div/div/form/div[3]/fieldset/div/div/div/div[{i}]/label'
    president1 = browser.find_element_by_xpath(absolutepath_president)
    president1.click()
    time.sleep(2)




SCROLL_PAUSE_TIME = 0.8


#slight changes to be made!!

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(browser.page_source, 'lxml')


In [8]:
import urllib3 
import pandas as pd
from datetime import datetime
import time
import string
import numpy as np



link_str = []

for link in soup.findAll('a', href=True):
    if link != None:
        link_str.append(link['href'])

# now I want the link strings containting /the-presidency/
new_link_str = []

for element in link_str:
    if element.find("/presidential-speeches/") >= 0: 
        new_link_str.append(element)
        


http = urllib3.PoolManager()
list2=[]
president_names=[]
for element in new_link_str:
    url = "http://millercenter.org" + element
#     print(element)
    response = http.request('GET', url)
    soup_2 = BeautifulSoup(response.data, 'lxml')
    list2.append(soup_2.findAll('div', {"class":"transcript-inner"}))
    president_names.append(soup_2.find('p', {"class":"president-name"}).get_text(strip=True))
    


#converting into string due to weird beautifulsoup output    
list3=[]
for i in range(len(list2)):
    if list2[i]:
        list3.append(str(list2[i]))
    else:
        
        #add comments for why I had to do it like this
        
        url = "http://millercenter.org" + new_link_str[i]
        response = http.request('GET', url)
        soup_2 = BeautifulSoup(response.data, 'lxml')
        list3.append(str(soup_2.findAll('div', {"class":"view-transcript"})))

# print(list3[1])

#this will be used to clean the text from html tags
def tag_cleanr(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

cleaned_speeches = []

#these replace lines could and should be replaced with regex since we covered that in the syllabus more extensively

for i in range(len(list3)):
    x = tag_cleanr(list3[i])
    x = x.replace('Transcript', '').replace('\r\n', '').replace('\n', '')
    cleaned_speeches.append(x)


#Get all the speech dates into a list

pattern = r'(\w+-\d+-\d{4})'
speech_dates=[]


for i in range(len(new_link_str)):
    match = re.findall(pattern, new_link_str[i])
    speech_dates.append(str(match))


for i in range(len(speech_dates)):
    #this feels like its terrible syntax but not sure if there is a nicer way, maybe I will replace it with regex
    
    speech_dates[i] = speech_dates[i].replace('[', '').replace(']', ''.replace("'", "")).replace("'", "")
    speech_dates[i] = datetime.strptime(speech_dates[i], '%B-%d-%Y')


df = pd.DataFrame({'dates':speech_dates, 'speeches':cleaned_speeches, 'president':president_names})

#this part is added because dataframe does not recognize [] as an empty input. 

df['speeches'].replace('[]', np.nan, inplace=True)

#now that I replaced [] with NaN, I can simply say drop all the rows with NaN elements

df.dropna(subset=['speeches'], inplace=True)


        
print(df)



KeyboardInterrupt: 

In [9]:
import pandas as pd

unpickled_df = pd.read_pickle("./FINAL SPEECH PRES SCORE DATA.pkl")
unpickled_df

,dates,speeches,president,positive,negative,compound
0,2016-05-15,[Hello Rutgers! (Applause.) R-U rah-rah! (A...,Barack Obama,0.470120,0.274480,0.228497
1,2016-03-22,[Thank you. (Applause.) Muchas gracias. Tha...,Barack Obama,0.387320,0.207360,0.297054
2,2016-01-12,"[Mr. Speaker, Mr. Vice President, Members of C...",Barack Obama,0.386680,0.288600,0.184648
3,2015-06-26,[THE PRESIDENT: Giving all praise and honor t...,Barack Obama,0.361520,0.186560,0.267056
4,2015-03-07,"[UDIENCE MEMBER: We love you, President Obama...",Barack Obama,0.256920,0.178680,0.233208
5,2015-01-20,"[Mr. Speaker, Mr. Vice President, Members of C...",Barack Obama,0.393000,0.265080,0.225514
6,2014-11-20,"[My fellow Americans, tonight, I’d like to tal...",Barack Obama,0.217920,0.159440,0.188497
7,2014-01-28,"[Mr. Speaker, Mr. Vice President, Members of C...",Barack Obama,0.322520,0.236200,0.295301
8,2013-12-04,"[Thank you. (Applause.) Thank you, everybody...",Barack Obama,0.294920,0.261360,0.198543
9,2013-09-10,"[My fellow Americans, tonight I want to talk t...",Barack Obama,0.186640,0.256480,-0.156255
